<img src="netflix.jpg">

# Importing Libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_csv(r"C:\Users\Raj\Desktop\NF Dataset\Machine-Learning\combined_data_1.txt",names = ['Cust_Id', 'Rating'])

In [3]:
data.shape

(24058263, 2)

In [12]:
df=data[:5000000]
df.shape

(5000000, 2)

In [13]:
arr=np.array(df.iloc[:,1])
x=np.argwhere(np.isnan(arr))
movie_np = []
movie_id = 1

for i in range(len(x)-1):
    temp = np.full((1,int(x[i+1]-x[i]-1)), movie_id)
    movie_np = np.append(movie_np, temp)
    movie_id+=1
    
last_record = np.full((1,len(df) - int(x[i+1]) - 1),movie_id)
movie_np = np.append(movie_np, last_record)
print(movie_np)
print('Movie numpy: {}'.format(movie_np))
print('Length: {}'.format(len(movie_np)))

[  1.   1.   1. ... 996. 996. 996.]
Movie numpy: [  1.   1.   1. ... 996. 996. 996.]
Length: 4999004


In [14]:
df = df[pd.notnull(df['Rating'])]

In [15]:
df['Movie_Id'] = movie_np.astype(int)

In [18]:
df['Cust_Id'] = df['Cust_Id'].astype(int)
print('-Dataset examples-')
print(df.head())
print(df.shape)

-Dataset examples-
   Cust_Id  Rating  Movie_Id
1  1488844     3.0         1
2   822109     5.0         1
3   885013     4.0         1
4    30878     4.0         1
5   823519     3.0         1
(4999004, 3)


In [ ]:
x=